# Multiclass classification: The [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset

In this tutorial, we will implement a simple convolutional network based on the VGG-16 architecture, which we will apply to multiclass image classification. We will use the CIFAR-10 dataset, a standard for this type of (elementary) problem.


In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Load the CIFAR-10 Dataset

The CIFAR-10 dataset consists of 60,000 color images from 10 classes, with 6,000 images per class. There are 50,000 training images and 10,000 test images. Several sample images are shown below, along with the class names.

<center><img src="https://production-media.paperswithcode.com/datasets/4fdf2b82-2bc3-4f97-ba51-400322b228b1.png"
             style="width:500px;"></center>
<caption><center><b> Figure 1: Some images of the database</b></center></caption>

In [ ]:
from tensorflow.keras.datasets import cifar10

Since the CIFAR-10 dataset is included in _TensorFlow_, we can load the dataset using the _load_data()_ function.

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
 
print(X_train.shape)
print(X_test.shape)

##### <i style="color:purple">**Exercise**: Display sample images from the dataset.</i>

It is always a good idea to inspect some images in a dataset. Display a sample image, with its associated label as a title. 

In [ ]:
# Defining array. Each item of array represent integer value of labels. 10 items for 10 labels.

class_names =['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

In [1]:
# %load solutions/cifar10/data_visualization.py
plt.figure(figsize=(18, 9))
 
n_rows = 4
n_cols = 8
 
# plot each of the images in the batch and the associated ground truth labels
for i in range(n_rows*n_cols):
    ax = plt.subplot(n_rows, n_cols, i + 1)
    plt.imshow(X_train[i,:,:])
    title = '['+y_train[i][0].astype('str')+'] '+class_names[y_train[i][0]]
    plt.title(title)
    plt.axis("off")
    
#plt.tight_layout()
plt.show()

Remember, the images in CIFAR-10 are quite small, only 32×32 pixels, so while they do not have much detail, there is still enough information in these images to support an image classification task.

##### <i style="color:purple">**Exercise**: Preprocess the data.</i>

* Normalize the image data to the range $[0,1]$.
* Convert the integer labels to one-hot encoded labels. Pour ce faire, on pourra utiliser la fonction [_to_categorical()_](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical).

In [ ]:
from tensorflow.keras.utils import to_categorical

In [2]:
# %load solutions/cifar10/data_preprocessing.py
# Normalize images to the range [0, 1].
X_train = X_train.astype("float32") / 255
X_test  = X_test.astype("float32") / 255
 
# Change the labels from integer to categorical data.
print('Original (integer) label for the first training sample: ', y_train[0])
 
# Convert labels to one-hot encoding.
y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)
 
print('After conversion to categorical one-hot encoded labels: ', y_train[0])

## CNN Model Implementation

This section will define a simple CNN model in Keras and train it on the CIRFAR-10 dataset.

The simple model we propose is shown in Figure 2. Its structure is similar to that of the VGG-16. However, it has fewer layers, and the size of the input images is much smaller, considerably reducing the number of trainable parameters. More specifically, this model contains three convolutional blocks and a fully connected layer - our classifier. 
The figure shows the number of filters at the end of each convolutional block, as well as the size of the convolution output images. 

To be more precise:
* The first convolutional block is composed of two convolutional layers with 32 filters each, followed by a max pooling layer with a stride of 2 so that the output shape from this first convolutional block is (16×16 x32). 
* The second convolutional block is nearly identical to the first but with 64 filters in each convolutional layer instead of 32. 
* Finally, the third convolutional block is an exact copy of the second one.

<center><img src="https://learnopencv.com/wp-content/uploads/2023/01/tensorflow-keras-cnn-architecture.png"
             style="width:800px;"></center>
<caption><center><b> Figure 2: CNN architecture</b></center></caption>
<br>

_**Note:** The number of filters in each convolutional layer is something that you will need to experiment with. A larger number of filters allows the model to have a greater learning capacity, but this also needs to be balanced with the amount of data available to train the model. Adding too many filters (or layers) can lead to overfitting, one of the most common issues encountered when training models._

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

##### <i style="color:purple">**Exercise**: Define the CNN architecture.</i>

Define a _cnn_model()_ function that instantiates the network described above.

In [3]:
# %load solutions/cifar10/cnn_model.py
def cnn_model(input_shape=(32, 32, 3)):
     
    model = Sequential()
     
    #------------------------------------
    # Conv Block 1: 32 Filters, MaxPool.
    #------------------------------------
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
 
    #------------------------------------
    # Conv Block 2: 64 Filters, MaxPool.
    #------------------------------------
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
 
    #------------------------------------
    # Conv Block 3: 64 Filters, MaxPool.
    #------------------------------------
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
     
    #------------------------------------
    # Flatten the convolutional features.
    #------------------------------------
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(10, activation='softmax'))
     
    return model

In [ ]:
model = cnn_model()
model.summary()

### Training

For training, you can use the following hyperparameters :
* `batch_size` = 256
* `epochs` = 30

##### <i style="color:purple">**Exercise**: Compile and train the model.</i>

You will use the _rmsprop_ optimizer, and track _accuracy_. For training, you can use the _validation_split_ agument to reserve 30% of the dataset for model validation.

In [4]:
# %load solutions/cifar10/cnn_train.py
batch_size = 256
epochs = 30

model.compile(optimizer = 'rmsprop', 
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'],
             )

history = model.fit(X_train,
                    y_train,
                    batch_size = batch_size, 
                    epochs = epochs, 
                    verbose = 1, 
                    validation_split = .3,
                   )

_**Note**: The _validation_split_ argument reserves the last % of the training dataset for validation. This approach is very convenient, but if the training data set has a specific order (for example, order by classes), you must take steps to randomize the order before proceeding with the split._

##### <i style="color:purple">**Exercise** : Visualize the evolution of metrics during training.</i>

You will write a function displaying the evolution of metrics during training, on the training and validation sets. Display accuracy and loss on separate figures.

In [5]:
# %load solutions/cifar10/plot_training_analysis.py
def plot_training_analysis():
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.subplot(1,2,1)
    plt.plot(epochs, acc, 'b', linestyle="--",label='Training accuracy')
    plt.plot(epochs, val_acc, 'g', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(epochs, loss, 'b', linestyle="--",label='Training loss')
    plt.plot(epochs, val_loss,'g', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_training_analysis()

The results from our baseline model reveal that the model is **overfitting**. Notice that the validation loss increases after about ten epochs of training while the training loss continues to decline. This means the network learns to model the training data well but does not generalize to unseen test data. The accuracy plot shows a similar trend where the validation accuracy levels off after about ten epochs while the training accuracy approaches 100% as training progresses. 

This is a common problem when training neural networks and can occur for several reasons. One reason is that the model can fit the nuances of the training dataset, especially when the training dataset is small.

## Adding dropout to the model

To help mitigate this problem, we can employ one or more regularization strategies to help the model generalize better. Regularization techniques help to restrict the model’s flexibility so that it does not overfit the training data.
Dropout is implemented in Keras as a particular layer type that randomly drops a percentage of neurons during training. When dropout is used in convolutional layers, usually after the max pooling layer, it eliminates a percentage of neurons in the feature maps. When used after a dense layer, a percentage of neurons in the fully connected layer are dropped.

##### <i style="color:purple">**Exercise** : Add dropout to previous CNN.</i>

Add a dropout layer at the end of each convolutional block and also after the first dense layer in the classifier. The input argument to the Dropout function is the fraction of neurons to (randomly) drop from the previous layer during the training process.

In [ ]:
from tensorflow.keras.layers import Dropout

In [6]:
# %load solutions/cifar10/cnn_model_dropout.py
def cnn_model_dropout(input_shape=(32, 32, 3)):
     
    model = Sequential()
     
    #------------------------------------
    # Conv Block 1: 32 Filters, MaxPool.
    #------------------------------------
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    #------------------------------------
    # Conv Block 2: 64 Filters, MaxPool.
    #------------------------------------
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    #------------------------------------
    # Conv Block 3: 64 Filters, MaxPool.
    #------------------------------------
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
     
    #------------------------------------
    # Flatten the convolutional features.
    #------------------------------------
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
     
    return model

In [ ]:
model = cnn_model_dropout()
model.summary()

##### <i style="color:purple">**Exercise**: Compile and train the model.</i>

Use the same parameters as previously.

In [7]:
# %load solutions/cifar10/cnn_train.py
batch_size = 256
epochs = 30

model.compile(optimizer = 'rmsprop', 
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'],
             )

history = model.fit(X_train,
                    y_train,
                    batch_size = batch_size, 
                    epochs = epochs, 
                    verbose = 1, 
                    validation_split = .3,
                   )

In [ ]:
plot_training_analysis()

In the graphs above, the training curves align very closely with the validation curves. Note also that we achieve higher validation accuracy than the baseline model, which contained no dropouts.

## Evaluate the model on the test dataset

We can now predict the results for all the test images, as shown in the code below.

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc*100:.3f}")

##### <i style="color:purple">**Exercise**: Evaluate the model.</i>

Write an _evaluate_model()_ function that, given a dataset and a model:
* Selects a batch of data,
* Determines the labels predicted by the model for this data,
* Displays the selected images, comparing the true and predicted labels as titles. The titles will be colored green when the two labels coincide and red when they do not. <br>
The [_plt.setp_](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.setp.html) function is used to change the style of an object via the `plt.setp(obj, style)` synthaxis.
* Counts the number of correctly predicted labels and displays the percentage of correct predictions.

In [8]:
# %load solutions/cifar10/evaluate_model.py
def evaluate_model(dataset, model):
 
    class_names =['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
    n_rows = 3
    n_cols = 6
     
    # Retrieve a number of images from the dataset.
    data_batch = dataset[0:n_rows*n_cols]
 
    # Get predictions from model.  
    predictions = model.predict(data_batch)
 
    plt.figure(figsize=(20, 8))
    n_matches = 0
         
    for idx in range(n_rows*n_cols):
        ax = plt.subplot(n_rows, n_cols, idx+1)
        plt.axis("off")
        plt.imshow(data_batch[idx])
 
        pred_idx = tf.argmax(predictions[idx]).numpy()
        truth_idx = np.nonzero(y_test[idx])
             
        title = str(class_names[truth_idx[0][0]]) + " | " + str(class_names[pred_idx])
        title_obj = plt.title(title, fontdict={'fontsize':13})
             
        if pred_idx == truth_idx:
            n_matches += 1
            plt.setp(title_obj, color='g')
        else:
            plt.setp(title_obj, color='r')
                 
        acc = n_matches/(idx+1)
    print("Prediction accuracy: ", int(100*acc)/100)
     
    return

In [ ]:
evaluate_model(X_test, model)

##### <i style="color:purple">**Exercise**: Visualize the confusion matrix.</i>

A confusion matrix is a very common metric used to summarize a classification problem's results. The information is presented as a table or matrix where one axis represents the ground truth labels for each class, and the other axis represents the predicted labels from the network. The entries in the table represent the number of instances from an experiment (sometimes represented as percentages rather than counts). Generating a confusion matrix in TensorFlow is accomplished by calling the _tf.math.confusion_matrix()_ function, which takes two required arguments: the list of ground truth labels and the associated predicted labels.

Be careful to compare labels that are only encoded in the same way (one-hot vs. label encoding !).

In [ ]:
import seaborn as sn

In [9]:
# %load solutions/cifar10/ConfusionMatrix.py
# Generate predictions for the test dataset.
predictions = model.predict(X_test)
 
# For each sample image in the test dataset, select the class label with the highest probability.
predicted_labels = [np.argmax(i) for i in predictions]

# Convert one-hot encoded labels to integers.
y_test_integer_labels = tf.argmax(y_test, axis=1)
 
# Generate a confusion matrix for the test dataset.
cm = tf.math.confusion_matrix(labels=y_test_integer_labels, predictions=predicted_labels)
 
# Plot the confusion matrix as a heatmap.
plt.figure(figsize=[14, 7])
sn.heatmap(cm, annot=True, fmt='d', annot_kws={"size": 12})
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

A confusion matrix is a content-rich representation of a model's class-level performance. It can be very instructive in better understanding the areas in which the model performs well and those in which it may have more difficulty. 

Here, for example, a few things stand out immediately. Two of the ten classes tend to be misclassified more than the others: Dogs and Cats. Specifically, when the input image is a cat (index 3), it is often misclassified as a dog, with 176 misclassified samples. When the input image is a dog (index 5), the most misclassified examples are cats, with 117 samples.
Note also that the last row, representing trucks, is most often confused with cars. All these observations make intuitive sense, given the classes' similarity.